## Вопрос ответный бот 

Для создания вопрос-ответных ботов на какую-то конкретную тему используют подход, называемый **Retrival-Augmented Generation**. Бот опирается на какую-то коллекцию текстов - базу знаний. Когда пользователь задаёт вопрос, ищутся тексты, похожие на его вопрос - и даются в качестве входного текста модели GPT.

![](images/hp_rag.svg)

Для создания таких ботов часто используется библиотека [LangChain](https://python.langchain.com). Для начала, установим `langchain` и сопутствующие библиотеки.

In [15]:
%pip install nltk==3.9.1 telebot==0.0.5 langchain==0.2.1 telebot sentence_transformers==3.2.1 langchain_community==0.2.4 langchain_chroma==0.1.4 unstructured yandex_chain==0.0.9 yandexcloud

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


## Википедия Гарри Поттера

В нашем примере мы будет строить бота на основе [Harry Potter Fandom Wiki](https://harrypotter.fandom.com/). Тексты из этой вики уже находятся в репозитории, их надо лишь разархивировать:


In [2]:
!7za x data/harry_text.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 10889554 bytes (11 MiB)

Extracting archive: data/harry_text.zip
--
Path = data/harry_text.zip
Type = zip
Physical Size = 10889554

Everything is Ok

Files: 6571
Size:       27144403
Compressed: 10889554


У нас в проекте появилась директория `content`, в которой лежат все текста. Вот как выглядит отдельно взятый текст:

In [6]:
with open('content/text/Белые искры.txt') as f:
    print(f.read().replace('\n\n','\n'))

## Белые искры

Белые искры
Информация о заклинании
Формула
Неизвестно

Тип
Чары

Эффект
Создаёт искры

Цвет
Белый
Белые искры (англ. White sparks) — чары, выпускающие сноп белых искр из кончика волшебной палочки. Если выпустить большое количество этих искр вблизи человека, ему станет трудно дышать и видеть. Его также можно использовать в дуэлях для ослепления соперника.
История[]
На матче по квиддичу между Сборной США и Сборной Лихтенштейна в 2014 году американские болельщики выпустили огромное количество синих, красных и белых искр, празднуя победу своей команды — это серьёзно затруднило видимость на стадионе.
За кулисами[]
Баубиллиус, возможно, является вербальной формулой этих чар, хотя это не подтверждено.
Появления[]
Wizarding World (Первое появление)
Гарри Поттер: Коллекционная карточная игра (Возможное появление)



Выполним несколько подготовительных действий...

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Разбиваем текст на фрагменты

Для поиска нам нужно будет выбирать достаточно небольшие фрагменты текста. Поэтому, если вдруг в базе есть большие тексты, из необходимо разбить на кусочки. Вот как это сделать:

In [1]:
import langchain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

chunk_size = 2048
chunk_overlap = 100
source_dir = "content/text"

loader = DirectoryLoader(
    source_dir, glob="*.txt", show_progress=True, recursive=True
)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
fragments = splitter.create_documents([x.page_content for x in loader.load()])
print(f"Количество кусочков = {len(fragments)}")

100%|██████████| 6571/6571 [03:32<00:00, 30.94it/s]


12613

### Сохраняем фрагменты в базу данных

Для поиска нужных фрагментов мы будем использовать хитрый подход - **эмбеддинги**. Каждому тексту будет сопоставляться некоторый вектор (набор числел), в соответствии с его смыслом. Мы дальше будем просто искать похожие вектора на наш запрос.

Вот как можно вычислить такой вектор для текста:

> Не забудьте добавить секрет **api_key** и **folder_id** в свой проект DataSphere!


In [2]:
from yandex_chain import YandexEmbeddings
import os

api_key = os.environ["api_key"]
folder_id = os.environ["folder_id"]

embeddings = YandexEmbeddings(folder_id=folder_id, api_key=api_key, sleep_interval=0.1)
vec = embeddings.embed_query("Hello, world!")
len(vec)

256

Теперь нам нужно проиндексировать все кусочки текстов

> **ВНИМАНИЕ**: Этот код выполняется долго (**20 минут**)! Выполняйте его только в том случае, если вы поменяете содержимое базы знаний, чтобы сделать своего бота!

In [4]:
from langchain_chroma import Chroma

db_dir = "content"

db = Chroma(persist_directory=db_dir, embedding_function=embeddings)

Чтобы просто посмотреть, как работает код, воспользуемся уже созданной ранее векторной базой данных и просто её разархивируем:

In [16]:
!7za x data/db.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 61684636 bytes (59 MiB)

Extracting archive: data/db.zip
--
Path = data/db.zip
Type = zip
Physical Size = 61684636

Everything is Ok

Files: 6
Size:       288220132
Compressed: 61684636


Подключимся к этой базе:

In [3]:
from langchain_chroma import Chroma

db_dir = "content"

vec_store = Chroma(persist_directory=db_dir, embedding_function=embeddings)

Теперь посмотрим, насколько хорошо находятся фрагменты текста по какому-то запросу:

In [5]:
q = "Какое заклинание помогает левитировать?"

retriever = vec_store.as_retriever(search_kwargs={"k": 5})
res = retriever.invoke(q)
for x in res:
    print("-" * 40)
    print(x.page_content)

----------------------------------------
## Левитация

Это незавершённая статья.Вы можете помочь проекту, исправив и дополнив её.

Левитация (англ. Levitation) — это магическая способность, позволяющая левитировать выбранные объекты.

Заклинания, используемые для левитации[]

Левикорпус

Левитационное заклинание

Левитационные чары

Локомотор

Мобилиарбус

Мобиликорпус

Парящие чары

Появления[]

Гарри Поттер и Философский камень (Первое появление)

Гарри Поттер и Философский камень (фильм)

Гарри Поттер и Философский камень (игра)

Гарри Поттер и Тайная комната

Гарри Поттер и Тайная комната (фильм)

Гарри Поттер и Тайная комната (игра)

Гарри Поттер и узник Азкабана

Гарри Поттер и узник Азкабана (фильм)

Гарри Поттер и узник Азкабана (игра)

Гарри Поттер и Кубок Огня

Гарри Поттер и Кубок огня (фильм)

Гарри Поттер и Кубок Огня (игра)

Гарри Поттер и Орден Феникса

Гарри Поттер и Орден Феникса (фильм)

Гарри Поттер и Орден Феникса (игра)

Гарри Поттер и Принц-полукровка

Гарри Потте

### Подключаемся к Yandex GPT

Для формирования целостного ответа на вопрос пользователей нам потребуется обработать найденные фрагменты текста с помощью Yandex GPT. 

In [7]:
from yandex_chain import YandexLLM,YandexGPTModel

instructions = """
Ты - дружелюбный чат-бот, отвечающий на вопросы про вселенную Гарри Поттера
"""

llm = YandexLLM(folder_id=folder_id,api_key=api_key, instruction_text=instructions, model=YandexGPTModel.Pro)

llm.invoke(q)

'Во вселенной Гарри Поттера существует множество заклинаний, которые позволяют левитировать или управлять полетом объектов и даже людей. Одно из основных заклинаний для левитации — это заклинание *«Вингардиум Левиоса»*.\n\nОднако стоит помнить, что большинство заклинаний во вселенной Гарри Поттера требуют определенного уровня подготовки и магической силы, чтобы их выполнить.'

В данном примере мы пока что никак не использовали наши текстовые документы.

### Собираем Retrieval-Augmented Generation

Пришла пора соединить всё вместе и научить бота отвечать на вопросы, подглядывая в нашу текстовую базу знаний . Для этого используем механизм цепочек (*chains*):

In [8]:
import langchain 
import langchain.prompts
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt = """
Ты - бот, умеющий разноваривать про вселенную Гарри Поттера. Пожалуйста, посмотри на 
текст ниже и ответь на вопрос, используя информацию из этого текста. Не надо писать про текст,
пиши просто ответ, но достаточно подробный.
Текст:
-----
{context}
-----
Вопрос:
{question}"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Создаём цепочку
chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = chain.invoke(q)
print(answer)

В мире Гарри Поттера есть несколько заклинаний, которые помогают левитировать предметы. Например, «Вингардиум Левиоса», «Левикорпус», «Локомотор» и другие.

Заклинание «Вингардиум Левиоса» часто используется для перемещения предметов по воздуху. С его помощью можно заставить объекты парить в воздухе. «Локомотор», как и его разновидность «Пиертотум Локомотор», также позволяет транспортировать предметы по воздуху, но эти перемещения надо контролировать волшебной палочкой. 

«Левикорпус» поднимает человека в воздух вверх ногами, а «Мобиликорпус» позволяет перемещать человека по воздуху в любом направлении. Также существует заклинание «Левиосо», которое заставляет предметы парить в воздухе, но при этом не позволяет их перемещать.


### Делаем телеграм-бота

Теперь сделаем телеграм-бота для ответа на вопросы про вселенную Гарри Поттера. Как делать телеграм-бота мы уже знаем из примеров в файле **Кирпичики**. Сначала надо создать телеграм-бота в телеграме (с помощью **botfather**), и запомнить его секретный ключ в секрете DataSphere `tg_token`.

In [10]:
import io,os
import telebot
from PIL import Image

telegram_token = os.environ['tg_token']

bot = telebot.TeleBot(telegram_token)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, который знает всё про вселенную Гарри Поттера. Спрашивай!')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    answer = chain.invoke(message.text)
    bot.send_message(message.chat.id, answer)
    
# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе
